# 데이터 셋 하나로 합치기

## 1. 데이터 불러오기

In [ ]:
import pandas as pd

customers = pd.read_csv('./data/olist_customers_dataset.csv')
geolocation = pd.read_csv('./data/olist_geolocation_dataset.csv')
orders = pd.read_csv('./data/olist_orders_dataset.csv')
order_items = pd.read_csv('./data/olist_order_items_dataset.csv')
order_payments = pd.read_csv('./data/olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('./data/olist_order_reviews_dataset.csv')
products = pd.read_csv('./data/olist_products_dataset.csv')
sellers = pd.read_csv('./data/olist_sellers_dataset.csv')
product_cat_name = pd.read_csv('./data/product_category_name_translation.csv')

## 2. geolocation 특성 파악 및 처리
- sellers 데이터 내 seller_zip_code_prefix와 customers 데이터 내 customer_zip_code_prefix는 서로 서로 다른 컬럼이므로 각각의 lat, lng 가 필요함
- 또한 geolocation 데이터 내에는 중복된 zip_code_prefix와 geolocation_state가 존재하므로 평균값으로 lat, lng 대치
- 따라서, geolocation 데이터 내에 해당되는 각 zip_code_prefix를 이용해 sellers와 customer의 새로운 lat, lng 컬럼 생성할 것

In [ ]:
geolocation = geolocation.rename(columns={'geolocation_zip_code_prefix':'zip_code_prefix'})
sellers = sellers.rename(columns={'seller_zip_code_prefix':'zip_code_prefix'})
customers = customers.rename(columns={'customer_zip_code_prefix':'zip_code_prefix'})

In [ ]:
sellers.seller_id.nunique(), len(sellers)

(3095, 3095)

In [ ]:
geolocation.zip_code_prefix.nunique(), len(geolocation)

(19015, 1000163)

In [ ]:
# zip_code_prefix가 중복되어있는지 확인하기 위함
geolocation.sort_values('zip_code_prefix')

,zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
1246,1001,-23.549292,-46.633559,sao paulo,SP
429,1001,-23.550498,-46.634338,sao paulo,SP
1182,1001,-23.549779,-46.633957,sao paulo,SP
1435,1001,-23.549292,-46.633559,sao paulo,SP
326,1001,-23.551427,-46.634074,sao paulo,SP
...,...,...,...,...,...
1000026,99980,-28.388679,-51.848863,david canabarro,RS
999950,99980,-28.388400,-51.845225,david canabarro,RS
1000002,99980,-28.388342,-51.845194,david canabarro,RS
999864,99990,-28.329472,-51.769109,muliterno,RS


In [ ]:
geolocation.isnull().sum()

zip_code_prefix      0
geolocation_lat      0
geolocation_lng      0
geolocation_city     0
geolocation_state    0
dtype: int64

#### lat, lng 평균값 적용

In [ ]:
geolocation2 = geolocation.pivot_table(index='zip_code_prefix', values=['geolocation_lat','geolocation_lng'], aggfunc='mean').reset_index()
geolocation2

,zip_code_prefix,geolocation_lat,geolocation_lng
0,1001,-23.550190,-46.634024
1,1002,-23.548146,-46.634979
2,1003,-23.548994,-46.635731
3,1004,-23.549799,-46.634757
4,1005,-23.549456,-46.636733
...,...,...,...
19010,99960,-27.953722,-52.025511
19011,99965,-28.183372,-52.039850
19012,99970,-28.343766,-51.874689
19013,99980,-28.389129,-51.843836


#### sellers와 geolocation 데이터 셋 merge 수행

In [ ]:
from functools import reduce

sellers_geo = pd.merge(sellers, geolocation2, on='zip_code_prefix', how='inner')
sellers_geo = sellers_geo.rename(columns={'zip_code_prefix':'seller_zip_code_prefix', 'geolocation_lat':'seller_geolocation_lat', 'geolocation_lng':'seller_geolocation_lng'})
sellers_geo

,seller_id,seller_zip_code_prefix,seller_city,seller_state,seller_geolocation_lat,seller_geolocation_lng
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,-22.893848,-47.061337
1,e0eabded302882513ced4ea3eb0c7059,13023,campinas,SP,-22.893848,-47.061337
2,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,-22.383437,-46.947927
3,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,-22.909572,-43.177703
4,1d2732ef8321502ee8488e8bed1ab8cd,20031,rio de janeiro,RJ,-22.909572,-43.177703
...,...,...,...,...,...,...
3083,f1fdf2d13186575751aa25876536d85c,5314,sao paulo,SP,-23.530647,-46.736453
3084,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR,-23.448041,-51.869960
3085,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP,-23.657851,-46.676925
3086,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS,-31.751072,-52.323202


#### customers와 geolocation 데이터 셋 merge 수행

In [ ]:
len(customers)

99441

In [ ]:
customers_geo = pd.merge(customers, geolocation2, on='zip_code_prefix', how='inner')
customers_geo = customers_geo.rename(columns={'zip_code_prefix':'customer_zip_code_prefix', 'geolocation_lat':'customer_geolocation_lat', 'geolocation_lng':'customer_geolocation_lng'})
customers_geo

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_geolocation_lat,customer_geolocation_lng
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,-20.498489,-47.396929
1,5dca924cc99eea2dc5ba40d11ec5dd0f,2761fee7f378f0a8d7682d8a3fa07ab1,14409,franca,SP,-20.498489,-47.396929
2,661897d4968f1b59bfff74c7eb2eb4fc,d06a495406b79cb8203ea21cc0942f8c,14409,franca,SP,-20.498489,-47.396929
3,702b62324327ccba20f1be3465426437,8b3d988f330c1d1c3332ccd440c147b7,14409,franca,SP,-20.498489,-47.396929
4,bdf997bae7ca819b0415f5174d6b4302,866755e25db620f8d7e81b351a15bb2f,14409,franca,SP,-20.498489,-47.396929
...,...,...,...,...,...,...,...
99158,0b7a30ba373aeb55cf28add5b5477956,8c8173e547e020f411aa55b2fceed861,87145,paicandu,PR,-23.501806,-52.150926
99159,ff6b440b2465bfaef05c675efb48cd15,e38a27b979a552e019e297ecc22dc67b,98860,eugenio de castro,RS,-28.527590,-54.145960
99160,3f7029d9b98a47370cd5f3a97adcbccd,5894d202cc869b4ec460fce3c6ed8cb5,5538,sao paulo,SP,-23.588483,-46.737908
99161,d9110683c7a282144e9fc97660026a28,5cbfdb85ec130898108b32c50d619c39,74980,aparecida de goiania,GO,-16.822735,-49.248175


## 3. 전체 데이터 합치기

In [ ]:
from functools import reduce

df1 = reduce(lambda x,y: pd.merge(x,y, on='order_id', how='inner'), [order_items, orders, order_payments, order_reviews])
df2 = reduce(lambda x,y: pd.merge(x,y, on='customer_id', how='inner'), [df1, customers_geo])
df3 = reduce(lambda x,y: pd.merge(x,y, on='product_id', how='inner'), [df2, products])
df4 = reduce(lambda x,y: pd.merge(x,y, on='seller_id', how='inner'), [df3, sellers_geo])
df5 = reduce(lambda x,y: pd.merge(x,y, on='product_category_name', how='inner'), [df4, product_cat_name])
df5

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,...,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,seller_geolocation_lat,seller_geolocation_lng,product_category_name_english
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,-22.496953,-44.127492,cool_stuff
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.90,17.96,e6eecc5a77de221464d1c4eaff0a9b64,delivered,2017-06-28 11:52:20,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,-22.496953,-44.127492,cool_stuff
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.90,18.33,4ef55bf80f711b372afebcb7c715344a,delivered,2018-05-18 10:25:53,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,-22.496953,-44.127492,cool_stuff
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.90,16.17,30407a72ad8b3f4df4d15369126b20c9,delivered,2017-08-01 18:38:42,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,-22.496953,-44.127492,cool_stuff
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.90,13.29,91a792fef70ecd8cc69d3c7feb3d12da,delivered,2017-08-10 21:48:40,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,-22.496953,-44.127492,cool_stuff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115032,2c12150c742ae2fa48bc703964c16c5f,1,2b54fb9a4acf707537eb53eb2458c241,ef728fa1f17436c91ed1ccd03dcf9631,2018-08-02 17:50:12,57.90,19.60,ab0cf72dfe0538a63a57d6905ccb7b57,delivered,2018-07-28 17:55:27,...,1000.0,25.0,25.0,25.0,17201,jau,SP,-22.294360,-48.558524,arts_and_craftmanship
115033,2ef4a11b6e24fdfbb43b92cb5f95edff,1,9c313adb4b38a55b092f53f83f78be9e,ef728fa1f17436c91ed1ccd03dcf9631,2018-07-30 09:17:39,19.00,12.86,ee1cfdc92e449920e25d3ca4ab4da4f6,delivered,2018-07-23 18:35:14,...,500.0,16.0,6.0,16.0,17201,jau,SP,-22.294360,-48.558524,arts_and_craftmanship
115034,2ef4a11b6e24fdfbb43b92cb5f95edff,2,eacb104882d39ffb53140b1d1860a7c3,ef728fa1f17436c91ed1ccd03dcf9631,2018-07-30 09:17:39,39.90,12.87,ee1cfdc92e449920e25d3ca4ab4da4f6,delivered,2018-07-23 18:35:14,...,500.0,21.0,7.0,14.0,17201,jau,SP,-22.294360,-48.558524,arts_and_craftmanship
115035,9f7ba492c273ffac8f90a623b8eadd3b,1,5aa0d6ed4e6c0995d62090cd82cde89a,ef728fa1f17436c91ed1ccd03dcf9631,2018-08-28 03:24:24,11.90,12.79,c61336d6c835632048f973d2f2db598d,delivered,2018-08-23 12:31:37,...,200.0,16.0,6.0,16.0,17201,jau,SP,-22.294360,-48.558524,arts_and_craftmanship


In [ ]:
df5.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'customer_id',
       'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'review_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'customer_geolocation_lat', 'customer_geolocation_lng',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'seller_geolocation_lat

In [ ]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115037 entries, 0 to 115036
Data columns (total 44 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       115037 non-null  object 
 1   order_item_id                  115037 non-null  int64  
 2   product_id                     115037 non-null  object 
 3   seller_id                      115037 non-null  object 
 4   shipping_limit_date            115037 non-null  object 
 5   price                          115037 non-null  float64
 6   freight_value                  115037 non-null  float64
 7   customer_id                    115037 non-null  object 
 8   order_status                   115037 non-null  object 
 9   order_purchase_timestamp       115037 non-null  object 
 10  order_approved_at              115023 non-null  object 
 11  order_delivered_carrier_date   113852 non-null  object 
 12  order_delivered_customer_date 

In [ ]:
df5.to_csv("olist_total.csv", index = False)

In [ ]:
pd.read_csv('olist_total.csv')

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,...,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,seller_geolocation_lat,seller_geolocation_lng,product_category_name_english
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,-22.496953,-44.127492,cool_stuff
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.90,17.96,e6eecc5a77de221464d1c4eaff0a9b64,delivered,2017-06-28 11:52:20,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,-22.496953,-44.127492,cool_stuff
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.90,18.33,4ef55bf80f711b372afebcb7c715344a,delivered,2018-05-18 10:25:53,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,-22.496953,-44.127492,cool_stuff
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.90,16.17,30407a72ad8b3f4df4d15369126b20c9,delivered,2017-08-01 18:38:42,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,-22.496953,-44.127492,cool_stuff
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.90,13.29,91a792fef70ecd8cc69d3c7feb3d12da,delivered,2017-08-10 21:48:40,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,-22.496953,-44.127492,cool_stuff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115032,2c12150c742ae2fa48bc703964c16c5f,1,2b54fb9a4acf707537eb53eb2458c241,ef728fa1f17436c91ed1ccd03dcf9631,2018-08-02 17:50:12,57.90,19.60,ab0cf72dfe0538a63a57d6905ccb7b57,delivered,2018-07-28 17:55:27,...,1000.0,25.0,25.0,25.0,17201,jau,SP,-22.294360,-48.558524,arts_and_craftmanship
115033,2ef4a11b6e24fdfbb43b92cb5f95edff,1,9c313adb4b38a55b092f53f83f78be9e,ef728fa1f17436c91ed1ccd03dcf9631,2018-07-30 09:17:39,19.00,12.86,ee1cfdc92e449920e25d3ca4ab4da4f6,delivered,2018-07-23 18:35:14,...,500.0,16.0,6.0,16.0,17201,jau,SP,-22.294360,-48.558524,arts_and_craftmanship
115034,2ef4a11b6e24fdfbb43b92cb5f95edff,2,eacb104882d39ffb53140b1d1860a7c3,ef728fa1f17436c91ed1ccd03dcf9631,2018-07-30 09:17:39,39.90,12.87,ee1cfdc92e449920e25d3ca4ab4da4f6,delivered,2018-07-23 18:35:14,...,500.0,21.0,7.0,14.0,17201,jau,SP,-22.294360,-48.558524,arts_and_craftmanship
115035,9f7ba492c273ffac8f90a623b8eadd3b,1,5aa0d6ed4e6c0995d62090cd82cde89a,ef728fa1f17436c91ed1ccd03dcf9631,2018-08-28 03:24:24,11.90,12.79,c61336d6c835632048f973d2f2db598d,delivered,2018-08-23 12:31:37,...,200.0,16.0,6.0,16.0,17201,jau,SP,-22.294360,-48.558524,arts_and_craftmanship
